## Распознавание аудио с помощью Cloud STT API

В большинстве случаев использование базовой модели распознавания является наилучшим выбором.

Ниже мы показываем, как использовать наше API для распознавания аудиозаписей прямо из Jupyter ноутбука.

### Перед началом

В первую очередь требуется склонировать репозиторий [Yandex.Cloud API](https://github.com/yandex-cloud/cloudapi) и сгенерировать код клиентской библиотеки. В данном случае предполагается, что репозиторий клонируется в корневую папку проекта в DataSphere.

In [ ]:
%pip install grpcio-tools

In [ ]:
#!:bash
python3 -m grpc_tools.protoc -I cloudapi -I cloudapi/third_party/googleapis --python_out=. --grpc_python_out=. google/api/http.proto google/api/annotations.proto yandex/cloud/api/operation.proto google/rpc/status.proto yandex/cloud/operation/operation.proto yandex/cloud/ai/stt/v2/stt_service.proto

Далее получите [IAM-токен](https://cloud.yandex.ru/docs/iam/operations/iam-token/create-for-sa) или [API-ключ](https://cloud.yandex.ru/docs/iam/operations/api-key/create) для вашего сервисного аккаунта. В примерах для аутентификации используется API-ключ.

Чтобы использовать IAM-токен, передавайте его в заголовках в следующем формате: `Authorization: Bearer <IAM-token>`

In [ ]:
API_KEY = '...'

Если при распознавании длинных аудио вы хотите загружать аудиозаписи в свой бакет прямо из датасферы, то вам также потребуется получить [статические ключи доступа](https://cloud.yandex.ru/docs/iam/concepts/authorization/access-key) для доступа к вашему Object Storage.

In [ ]:
AWS_ACCESS_KEY_ID = '...'
AWS_SECRET_ACCESS_KEY = '...'

### Потоковое распознавание аудио

**Замечание:** Приведённый ниже код потокового распознавания является немного упрощённой версией кода, представленного [здесь](https://cloud.yandex.ru/docs/speechkit/stt/streaming).

Функция `gen_chunks` реализует генерацию чанков, которые затем посылаются в потоковое распознавание. 

Также в целях упрощения мы получаем только финальный результат распознавания (флаг `partial_results` выставлен в `False`).

In [9]:
import io
import json
import grpc
import uuid
import yandex.cloud.ai.stt.v2.stt_service_pb2 as stt_service_pb2
import yandex.cloud.ai.stt.v2.stt_service_pb2_grpc as stt_service_pb2_grpc

from pathlib import Path
from tqdm import tqdm
from pydub import AudioSegment

In [10]:
CHUNK_SIZE = 4000


def gen_chunks(audio_file: Path):
    ##
    #  Читаем аудиозапись и сохраняем её в объект AudioSegment
    ##
    with audio_file.open('rb') as fp:
        audio = AudioSegment.from_wav(fp)
    
    ##
    #  Создаём конфиг, в спецификации указываем тип используемой модели, а также формат передаваемой записи
    ##
    specification = stt_service_pb2.RecognitionSpec(
        language_code='ru-RU',
        profanity_filter=False,
        model='general',
        partial_results=False,
        audio_encoding='LINEAR16_PCM',
        sample_rate_hertz=audio.frame_rate,
        raw_results=True
    )
    streaming_config = stt_service_pb2.RecognitionConfig(specification=specification)

    ##
    #  В качестве первого чанка отправляем конфиг
    ##
    yield stt_service_pb2.StreamingRecognitionRequest(config=streaming_config)
    
    try:
        ##
        #  Далее отправляем чанки записи без WAV заголовка
        ##
        raw_data = io.BytesIO(audio.raw_data)
        chunk = raw_data.read(CHUNK_SIZE)
        while chunk != b'':
            yield stt_service_pb2.StreamingRecognitionRequest(audio_content=chunk)
            chunk = raw_data.read(CHUNK_SIZE)                
    except Exception as e:
        print(f'Failed to load {audio_file}: {str(e)}')

In [11]:
def recognize_streaming(api_key, audio_file):
    ##
    #  Создаём grpc канал для отправки запроса на распознавание серверу
    ##
    req_id = str(uuid.uuid4())
    cred = grpc.ssl_channel_credentials()
    channel = grpc.secure_channel('stt.api.cloud.yandex.net:443', cred)
    stub = stt_service_pb2_grpc.SttServiceStub(channel)

    try:
        ##
        #  Отправляем запрос на сервер. В данном случае для получения доступа используется сервисный ключ api_key.
        #  Результатом запроса является стрим (итератор) с промежуточными распознаваниями передаваемой записи.
        ##
        it = stub.StreamingRecognize(gen_chunks(audio_file), metadata=(
            ('authorization', 'Api-Key ' + api_key),
            ('x-client-request-id', req_id),
            ('x-normalize-partials', 'false'),
            ('x-sensitivity-reduction-flag', 'false')
        ))
        return ' '.join(chunk.alternatives[0].text for r in it for chunk in r.chunks)
    except grpc._channel._Rendezvous as err:
        print(f'Failed to recognize {audio_file}. Error code {err._state.code}, message: {err._state.details}')
    except Exception as err:
        print(f'Failed to recognize {audio_file}: {err}')

In [12]:
audio_dir = Path('examples', 'speechkitpro', 'recognize_api', 'audio')

for audio_file in audio_dir.iterdir():
    if not audio_file.name.endswith('.wav'):
        continue
    recognition = recognize_streaming(API_KEY, audio_file)
    print(f'{audio_file.name}: {recognition}')

1.wav: пусть этом никто не сомневается
3.wav: это гром заседание высокого уровня будет принят краткий итоговый документ
2.wav: желаем вам всяческих успехов руководстве работы нашего комитета


### Распознавание длинных аудио

**Замечание:** Приведённый ниже код распознавания длинных аудио основан на [этом](https://cloud.yandex.ru/docs/speechkit/stt/transcribation) туториале. 

In [13]:
import time
import requests
import boto3


def upload_object_s3(aws_access_key_id, aws_secret_access_key, bucket, object_name, audio_io):
    s3 = boto3.client(
        service_name='s3',
        endpoint_url='https://storage.yandexcloud.net', 
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key
    )
    s3.upload_fileobj(audio_io, bucket, object_name)
    return s3.generate_presigned_url('get_object', Params={'Bucket': bucket, 'Key': object_name}, ExpiresIn=300)


def recognize_transcribation(api_key, aws_access_key_id, aws_secret_access_key, audio_file):
    ##
    #  Читаем аудиозапись и сохраняем её на S3, предварительно избавившись от WAV заголовка
    ##
    with audio_file.open('rb') as fp:
        audio = AudioSegment.from_wav(fp)
    audio_uri = upload_object_s3(aws_access_key_id, aws_secret_access_key, 'audio-examples', audio_file.name, io.BytesIO(audio.raw_data))
    
    ##
    #  Создаём grpc канал для отправки запроса на распознавание серверу
    ##
    req_id = str(uuid.uuid4())
    cred = grpc.ssl_channel_credentials()
    channel = grpc.secure_channel('transcribe.api.cloud.yandex.net:443', cred)
    stub = stt_service_pb2_grpc.SttServiceStub(channel)

    ##
    #  В спецификации указываем тип используемой модели, а также формат передаваемой записи
    ##
    specification = stt_service_pb2.RecognitionSpec(
        language_code='ru-RU',
        profanity_filter=False,
        model='general',
        partial_results=False,
        audio_encoding='LINEAR16_PCM',
        sample_rate_hertz=audio.frame_rate,
        audio_channel_count=audio.channels,
        raw_results=True,
    )
    
    ##
    #  Формируем финальный запрос на распознавание
    ##
    recognition_config = stt_service_pb2.RecognitionConfig(specification=specification)
    audio = stt_service_pb2.RecognitionAudio(uri=audio_uri)
    request = stt_service_pb2.LongRunningRecognitionRequest(config=recognition_config, audio=audio)

    try:
        ##
        #  Отправляем запрос на сервер. В данном случае для получения доступа используется сервисный ключ api_key
        ##
        response = stub.LongRunningRecognize(request, metadata=(
            ('authorization', 'Api-Key ' + api_key),
            ('x-client-request-id', req_id),
            ('x-normalize-partials', 'false'),
            ('x-sensitivity-reduction-flag', 'false')
        ))
        
        while True:
            time.sleep(1)

            ##
            #  Поскольку ответом на предыдущий запрос является id операции, осуществляющей распознавание,
            #  в цикле проверяем, не закончилось ли выполнение операции с заданным id, и если закончилось,
            #  то возвращаем полученное распознавание
            ##
            GET = "https://operation.api.cloud.yandex.net/operations/{id}"
            req = requests.get(GET.format(id=response.id), headers={'Authorization': 'Api-Key ' + api_key})
            req = req.json()

            if req['done']:
                return ' '.join(chunk['alternatives'][0]['text'] for chunk in req['response']['chunks'])
                
    except Exception as err:
        print(f'Failed to recognize {audio_file}: {err}')

In [14]:
audio_dir = Path('examples', 'speechkitpro', 'recognize_api', 'audio')

for audio_file in audio_dir.iterdir():
    if not audio_file.name.endswith('.wav'):
        continue
    recognition = recognize_transcribation(API_KEY, AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, audio_file)
    print(f'{audio_file.name}: {recognition}')

1.wav: пусть в этом никто не сомневается
3.wav: итогом заседании высокого уровня будет принят краткий итоговый документ
2.wav: желаем вам всяческих успехов в руководстве работы нашего комитета


### Распознавание коротких аудио

**Замечание:** Приведённый ниже код распознавания коротких аудио основан на [этом](https://cloud.yandex.ru/docs/speechkit/stt/request) туториале. 

In [15]:
import requests
import json


def recognize_short_audio(api_key, audio_file):
    ##
    #  Читаем аудиозапись и сохраняем её в объект AudioSegment
    ##
    with audio_file.open('rb') as fp:
        audio = AudioSegment.from_wav(fp)
    
    ##
    #  Определяем параметры запроса, указав параметры модели и формат передаваемой записи
    ##
    params = '&'.join([
        'topic=general',
        'lang=ru-RU',
        f'sampleRateHertz={audio.frame_rate}',
        'format=lpcm'
    ])
    POST = 'https://stt.api.cloud.yandex.net/speech/v1/stt:recognize?' + params
    
    try:
        ##
        #  Делаем POST запрос на распознавание аудио, сам аудиофрагмент передаём без WAV заголовка
        ##
        req = requests.post(POST, data=audio.raw_data, headers={'Authorization': 'Api-Key ' + api_key})
        req = req.json()
        return req['result']
    except Exception as err:
        print(err)

In [16]:
for audio_file in audio_dir.iterdir():
    if not audio_file.name.endswith('.wav'):
        continue
    recognition = recognize_short_audio(API_KEY, audio_file)
    print(f'{audio_file.name}: {recognition}')

1.wav: Пусть в этом никто не сомневается
3.wav: Итогом заседании высокого уровня будет принят краткий итоговый документ
2.wav: Желаем вам всяческих успехов в руководстве работы нашего комитета
